<center>_by Aaron Parsons_</center>

# Purpose

The purpose of this memo is to define the time grid in local sidereal time (LST) to which we will lock integrations from the HERA Correlator. In doing so, we also define the maximum integration time for the correlator employing baseline-dependent averaging (BDA). By locking integrations to an LST grid, we avoid having to interpolate between integrations when stacking or comparing days of observation.

This memo supports an upgrade to the HERA Correlator control between the H4C and H5C seasons. Prior to this memo, LST grid alignment was achieved by attempting to find a GPS-derived 1PPS (one pulse per second) close to an LST grid boundary and arming F-engines to reset their master counters (MCNTs) on that 1PPS. This system had the shortcomings that
* the actual LST grid of the observations depended on run-time operation (i.e. it emerged from the start time rather than being pre-defined),
* little attention was given to how evenly the chosen LST grid divided a sidereal day, and
* there was no support for LST alignment under BDA.

The goal of the new system defined by this memo is to address these shortcomings and clarify the control path for implementing LST synchronization in the correlator.

# Background

The HERA correlator consists of three major subsystems: the F-Engine responsible for synchronization and channelization, the X-Engine responsible for cross-correlation and accumulation, and the data recorder that writes accumulations and meta-data to files on disk.

F-Engine synchronization is achieved by distributing phase-aligned sample clocks and 1PPS signals to each HERA SNAP board using a network-based synchronization system known as White Rabbit. F-Engines re-synchronize on the next 1PPS signal when an 'arm' command is sent. The timing of 1PPS relative to the LST grid is arbitrary.

On the rising edge of the next 1PPS, they reset the MCNT counter which count the number of subsequent spectra. This counter is embedded in the packets output by the F-Engine and is used by the network receive buffer on the GPU-accelerated X-Engines to time-align spectra for cross-correlation.

After cross-correlation, visibility products are accumulated on the X-Engine for an integer number of MCNTs, as specified in configuration. We will call this parameter $MCNT\_PER\_GRID$. In the current (non-BDA) correlator, all accumulation buffers use this same parameter. After transitioning to BDA, the correlator will support power of 2 subdivisions of this parameter for baselines, as appropriate for their length, up to $2^4$.

For the purpose of BDA, we usually concern ourselves with the minimum number of MCNTs per integration, but for defining an LST grid for the correlator, we are actually more concerned with the maximum integration time, since this defines the coarsest grid for alignment. Thus, for this purpose of this memo, $MCNT\_PER\_GRID$ is taken to be correspond to the longest integration time for which we support LST alignment.

# Interfaces and Definitions

Correlator control and communication is implemented primarily through a <tt> redis</tt> database interface accessible from devices internal to the correlator. We adopt the philosophy that the <tt>redis</tt> database is the authority of record for understanding the state of the correlator. Control scripts and configuration files may be used to command changes, but changes are not valid unless they are updated to <tt>redis</tt>. The information contained in <tt>redis</tt> should be sufficiently detailed to fully characterize the operation of the correlator.

In this vein, we will enumerate below the <tt>redis</tt> keys communicating the correlator state.

## F-Engine

Configuration of the SNAP-based F-Engines is controlled via <tt>hera_corr_f/scripts/hera_corr_feng_init.py</tt>. 
This script accesses <tt>redis</tt> through a python interface.
To support LST grid alignment, this script must record the sample clock frequency and the synchronization time to <tt>redis</tt>.
<table>
   <tr>
       <td><b>Parameter</b></td>
       <td><b>Description</b></td>
       <td><b><tt>redis</tt> key</b></td>
       <td><b>Set by</b></td>
   </tr>
   <tr>
       <td>$N_{\rm samp}$</td>
       <td>Samples per MCNT</td>
       <td><tt>feng:samples_per_mcnt</tt></td>
       <td><tt>hera_corr_feng_init.py</tt></td>
   </tr>
   <tr>
       <td>$f_{\rm samp}$</td>
       <td>Sample clock frequency in Hz</td>
       <td><tt>feng:sample_freq</tt></td>
       <td><tt>hera_corr_feng_init.py</tt></td>
   </tr>
   <tr>
       <td>$t_{\rm sync}$</td>
       <td>Synchronization time in UTC seconds </td>
       <td><tt>feng:sync_time</tt></td>
       <td><tt>hera_corr_feng_init.py</tt></td>
   </tr>
   <tr>
       <td>corr_map</td>
       <td>Mapping of antenna number to correlator index</td>
       <td><tt>corr:map</tt></td>
       <td><tt>hera_upload_config_to_redis.py</tt> (with <tt>hera_feng_config.yaml</tt>)</td>
   </tr>
</table>

The number of samples per MCNT, $N_{\rm samp}$ (which for HERA is the number of samples in a spectrum), is compiled into the FPGA design of the SNAP.

$t_{\rm sync}$ and $f_{\rm samp}$ are both configurable at run time (although $f_{\rm samp}$ must not violate the maximum clock rate for which the FPGA design is compiled. Since it is the job of <tt>hera_corr_feng_init.py</tt> to configure these for each SNAP, it reports the values to <tt>redis</tt>.
These three parameters are used by the X-Engine to determine the starting MCNT that aligns integration windows to the LST Grid.

The mapping of antenna number to correlator input is manipulated by uploading a yaml file to redis. Because BDA
configuration depends on this mapping, uploading a new yaml should be accompanied by erasure of the <tt>redis</tt> BDA_CONFIG key to avoid <tt>redis</tt> becoming internally inconsistent. Alternately (for the future), antenna mapping and BDA configuration could be moved out of their respective subsystems into a single yaml governing correlator configuration.

## X-Engine

The X-Engine consists of a <tt>hashpipe</tt> pipeline connecting a network device to shared memory, and the <tt>xGPU</tt> code base for loading from that shared memory onto a GPU cross-correlation engine, integrating, and then retrieving the results back to the CPU for <tt>px[0-15]</tt>. The <tt>hashpipe_redis_gateway</tt> daemon is responsible for passing the state of the <tt>redis</tt> database into a <tt>hashpipe</tt> instance on each <tt>px[0-15]</tt> machine. The relevant code is in <tt>paper_gpu</tt>, with <tt>xGPU</tt> and <tt>hashpipe</tt> as a subrepos.

<table>
    <tr>
        <td><b>Parameter</b></td>
        <td><b>Description</b></td>
        <td><b><tt>redis</tt> key</b></td>
        <td><b>Set by</b></td>
    </tr>
   <tr>
       <td>NTIME</td>
       <td>Integer MCNTs per GPU block</td>
       <td><tt>xeng:ntime</tt></td>
       <td><tt>xtor_up</tt> (from <tt>xgpu_info.h</tt>)</td>
   </tr>
   <tr>
       <td>INTSTART</td>
       <td>Integer MCNT to start integrating</td>
       <td><tt>xeng:intstart</tt></td>
       <td><tt>hera_ctl.py</tt></td>
   </tr>
   <tr>
       <td>PKTS_PER_BASELINE?</td>
       <td>Integer packets transmitting a baseline subspectrum</td>
       <td><tt>xeng:pkts_per_baseline</tt></td>
       <td><tt>xtor_up</tt> (from <tt>xgpu_info.h</tt>)</td>
   </tr>
   
</table>

NTIME is a compile-time parameter set in <tt>xgpu_info.h</tt>. It is tuneable, with the constraint that it
be divisible by 16. The value of this parameter sets the minimum integration time in the GPU, as well as the resolution for synchronizing integrations to an LST grid.

INTSTART determines the time alignment of integrations in the correlator, and is derived from the current Local Sidereal Time (LST; itself a function of current time and LONGITUDE), NGRID, $t_{\rm sync}$, $f_{\rm samp}$, and $N_{\rm samp}$. It is computed from these values in <tt>redis</tt>, written back to <tt>redis</tt>, and passed from there to <tt>hashpipe</tt> via the <tt>hashpipe_redis_gateway</tt>.

PKTS_PER_BASELINE determines how many packets are needed to transmit the accumulated visibilities from a xGPU instance, per baseline. This factors into the computation of BCNTS_PER_FILE, the total number of packets that correspond to a file in the Data Catcher.

## Data Catcher

The Data Catcher code lives in the <tt>paper_gpu</tt> repository and is responsible for CPU integration on the <tt>px[0-15]</tt> machines, the transmission of the accumulated products to the <tt>hera-sn1</tt> (data catcher) machine. The Data Catcher system is responsible for BDA, and sets various <tt>redis</tt> keys that are communicated to the X-Engines.

<table>
    <tr>
        <td><b>Parameter</b></td>
        <td><b>Description</b></td>
        <td><b><tt>redis</tt> key</b></td>
        <td><b>Set by</b></td>
    </tr>
   <tr>
       <td>NGRID</td>
       <td>Integer number of LST bins for start alignment.</td>
       <td><tt>catcher:ngrid</tt></td>
       <td><tt>hera_create_bda_config.py</tt></td>
   </tr>
   <tr>
       <td>BDA_CONFIG</td>
       <td>Integer number of GPU blocks per baseline integration</td>
       <td><tt>catcher:bda_config</tt></td>
       <td><tt>hera_create_bda_config.py</tt> (from antpos/connections)</td>
   </tr>
   <tr>
       <td>LATITUDE</td>
       <td>Latitude of the array</td>
       <td><tt>catcher:latitude</tt></td>
       <td><tt>hera_create_bda_config.py</tt></td>
   </tr>
   <tr>
       <td>LONGITUDE</td>
       <td>Longitude of the array (used to compute LST)</td>
       <td><tt>catcher:latitude</tt></td>
       <td><tt>hera_create_bda_config.py</tt></td>
   </tr>
   <tr>
       <td>BCNTS_PER_FILE</td>
       <td>Integer number of packets <tt>hera_catcher_disk_thread_bda.c</tt> expects</td>
       <td><tt>catcher:bcnt_max</tt></td>
       <td><tt>hera_catcher_up.py</tt></td>
   </tr>
   <tr>
       <td>NBL{2,4,8,16}SEC (deprecated?)</td>
       <td>Integer number of baselines with {2,4,8,16} integrations</td>
       <td><tt>catcher:nbl{2,4,8,16}sec</tt></td>
       <td><tt>hera_gpu_bda_thread.c</tt></td>
   </tr>
   <tr>
       <td>NBDABLS (deprecated?)</td>
       <td>Number of baselines
       <td><tt>catcher:nbl{2,4,8,16}sec</tt></td>
       <td><tt>hera_gpu_bda_thread.c</tt></td>
   </tr>
</table>

The antenna/array position dependence of BDA is handled by <tt>hera_create_bda_config.py</tt>, with input parameters handeled by a TBD config file that is uploaded into <tt>redis</tt>. This config file specifies
the thresholds for assigning baselines to one of four integration tiers [1, 2, 3, 4] on the basis of the baseline vector. 
It also specifies NGRID parameter that defines how many bins a sidereal day is divided into, which in turn determines the boundaries to which correlator integrations are to be synchronized.

BDA_CONFIG is a plain text format of the form "ANT_I ANT_J TIER\n" mapping a pairing of correlator indices ANT_I and ANT_J to a TIER of integration. Multiple entries are separated by a newline. Because this mapping depends on the mapping of antenna number to correlator index, BDA_CONFIG should be erased if a new corr_map (derived from an uploaded <tt>hera_feng_config.yaml</tt> in <tt>redis</tt>) is written in.

LONGITUDE is necessary for calculating LST for INTSTART in the X-Engine. It should be written to <tt>redis</tt>
by <tt>hera_create_bda_config.py</tt>, since this is where position information is to be held.

BCNTS_PER_FILE depends on PKTS_PER_BASELINE, as well as the number of baselines falling into each integration tier in BDA_CONFIG. It tells <tt>hera_catcher_disk_thread_bda.c</tt> how many packets to expect each integration.

# Architecture and Constraints

Under the proposed system, an absolute LST grid is pre-defined by dividing a sidereal day into an integer number of bins, each spanning a time of
\begin{equation}
t_{\rm grid} \equiv t_{\rm sidereal} / NGRID
\end{equation}
while simultaneously satisfying
\begin{equation}
t_{\rm grid} \approx MCNT\_PER\_INT \cdot N_{\rm samp} / f_{\rm clk},
\end{equation}
where $N_{\rm samp}$ is the number of time samples used to compute one spectrum in the F-Engine (nominally $2^{14}$) and $f_{\rm clk}$ is the sample clock (nominally 500 MHz).

Because $t_{\rm grid}$ as realized in the second equation above may only approximately divide $t_{\rm sidereal}$, there will be drift of the correlator's integration boundaries with respect to our absolute LST grid. We will take the LST grid indices to be defined by an even division of $[0,2\pi)$ into $NGRID$ bins:
\begin{equation}
LST\_GRID\_INDEX \equiv {\rm floor}\left[NGRID\cdot\frac{LST}{2\pi}\right].
\end{equation}

Our goal is to find a value for $MCNT\_PER\_INT$ such the $LST\_GRID\_INDEX$ can be calculated as accurately as possible using integer arithmetic on $MCNT$. We will do this by choosing an $MCNT\_PER\_INT$ that closely matches $t_{\rm grid}$, and by choosing $INTSTART$, the MCNT that aligns with the edges of our predetermined LST bins.
When the correlator is synchronized (each day, to avoid accumlating drift), any arbitrary 1PPS event will be used; the synchronization script records the $t_{\rm sync}$ that can be used to choose the appropriate
$INTSTART$.

Thus, $LST\_GRID\_INDEX$ may be approximated inside the correlator as 
\begin{equation}
LST\_GRID\_INDEX \approx
{\rm floor}\left[\frac{MCNT - INTSTART}{MCNT\_PER\_INT}\right]~\%~NGRID
\end{equation}
with integration boundaries at
\begin{equation}
(MCNT - INTSTART)~ \%~ MCNT\_PER\_INT == 0
\end{equation}

## Additional Constraints

Internal correlator buffering and our choices for BDA integration times present additional constraints on $NGRID$ and $MCNT\_PER\_INT$.

The number of spectra spanned by an LST grid cell
($MCNT\_PER\_INT$) should be divisible by 32, supporting grid subdivisions for BDA while ensuring alignment in LST.

Also, X-Engines bundle $NTIME$ spectra together, we require that $MCNT\_PER\_INT$ be additionally divisible by $NTIME$, which in turn must be divisible by 16.
Thus, $2^5\cdot2^{4}=2^{9}$ must divide $MCNT\_PER\_INT$ evenly.

Finally, we expect the maximum integration time for BDA to be in the neighborhood of 16 s, but in order to have head room, we will adopt a target value of
\begin{equation}
t_{\rm grid}\approx 32 {\rm s}.
\end{equation}
The cost of this headroom is just an added 16 s of wait-time for correlator synchronization.

# Numerology

Assuming a sample clock of $f_{\rm samp}=500$ MHz, a real-valued FFT input of $N_{\rm samp}=16384=2^{14}$ samples, and correlator blocks that bundle $NTIME=2048$ spectra per packet (current default),
let us first compute the number of blocks in a sidereal day, which defines the fundamental time resolution of correlator integrations.

In [69]:
t_sidereal = 86164.0905 # s / sidereal day
f_clk = 500e6 # Hz
NSAMP = 2**14 # samples per spectrum (8192 ch, real-sampled)
# X-Engine code says NTIME (via NTIME_PIPE in xGPU/xgpu_info.h) must be divisible by 16 for DP4A
NTIME_DEFAULT = 2**11 # current default spectra per X-engine block (fundamental GPU resolution)
#NTIME = 16 * 157

In [72]:
t_mcnt = 1 / f_clk * NSAMP
t_blk = t_mcnt * NTIME_DEFAULT
mcnt_per_lst = t_sidereal / t_mcnt
blk_per_lst = t_sidereal / t_blk
t_error = t_sidereal - int(round(blk_per_lst)) * t_blk
print('MCNT per sidereal day:  ', mcnt_per_lst)
print('Blocks per sidereal day:  ', blk_per_lst)
print('Fundamental drift wrt LST grid [s]:  ', t_error)

MCNT per sidereal day:   2629519363.4033203
Blocks per sidereal day:   1283945.0016617775
Fundamental drift wrt LST grid [s]:   0.00011151999933645129


As the calculations above show, given a sample clock of 500 MHz, we do not have an exact integral number of packets per sidereal day, and with the additional default blocking of 2048 in the X-Engine, this leads to a $\sim111\mu s$ drift from periodicity per day for continuous operation.

Unfortunately, we must incur even more drift because we must factor <tt>mcnt_per_lst</tt> into $NGRID\cdot MCNT\_PER\_INT$, with the constraint that $MCNT\_PER\_INT$ be divisible by $2^9$. Note that the closest 
integer number of MCNTs per sidereal day is not even even.

To get $t_{\rm grid}$ near 32 s, we need $NGRID$ in the range of 2400 to 3000, and we need to pair it with an $MCNT\_PER\_INT$ that
is divisible by $2^{9}$ (or an $NBLK\_PER\_INT$ divisible by $2^5$) such that the product is near to the numbers above.
\begin{align*}
NGRID&\in[2400, 3000],\\
MCNT\_PER\_INT~ \%~ 2^{16} &= 0,\\
NGRID\cdot MCNT\_PER\_INT &\approx 1283945.
\end{align*}

What follows is a brute-force search for such a pairing, with $t_{\rm error}$ corresponding to the drift, in seconds, relative to our 
LST grid after a continuous day of operation.

In [73]:
print('t_grid\tNGRID\tNblk_PER_INT\tt_error')
for ngrid in range(2400, 3000):
    t = t_sidereal / ngrid
    nblk_per_int = int(round(blk_per_lst / ngrid))
    if nblk_per_int % 2**5 == 0:
        print('%6.2f\t%5d\t%12d\t%+6.3f' % \
              (t, ngrid, nblk_per_int,
               t_sidereal - t_blk * ngrid * nblk_per_int))

t_grid	NGRID	Nblk_PER_INT	t_error
 34.38	 2506	         512	+58.586
 34.37	 2507	         512	+24.226
 34.36	 2508	         512	-10.133
 34.34	 2509	         512	-44.493
 34.33	 2510	         512	-78.853
 32.23	 2673	         480	+60.734
 32.22	 2674	         480	+28.521
 32.21	 2675	         480	-3.691
 32.20	 2676	         480	-35.903
 32.19	 2677	         480	-68.115
 30.10	 2863	         448	+88.651
 30.09	 2864	         448	+58.586
 30.07	 2865	         448	+28.521
 30.06	 2866	         448	-1.543
 30.05	 2867	         448	-31.608
 30.04	 2868	         448	-61.673
 30.03	 2869	         448	-91.738


# Results

The result above that skews the least from periodicity (${\rm min}~ t_{\rm error} $) while remaining close to $t_{\rm grid}\approx32$ s
is
\begin{align*}
NGRID &= 2675,\\
NPKT\_PER\_INT &= 480,
\end{align*}
which skews -3.39 s from periodicity for each continuous day of operation. This corresponds to an integration time of $t_{\rm grid}\approx32.21$ s.

Given that our fastest integration times are anticipated to be $\sim 1$ s, this is probably an acceptable amount of drift. Still, we should try to ensure that we re-synchronize the correlator each observing day.

Thus, we adopt an LST grid of $NGRID=2675$ bins, each 31.61 s wide, indexed to 0 at $LST=LST_0=18h00$. Inside the correlator, this corresponds to 480 packets per integration, with each packet containing 4 spectra, giving us a value of
\begin{equation}
MCNT\_PER\_INT=983040.
\end{equation}

What follows is example code for supporting this choice of grid.

In [57]:
import numpy as np

t_sidereal = 86164.0905 # s / sidereal day

NGRID = 2726
#NBLK_PER_INT = 120576
NBLK_PER_INT = 384
#NGRID = 2675
#NBLK_PER_INT = 480
#NSPEC_PER_BLK = 2**11
NSPEC_PER_BLK = 16 * 157
MCNT_PER_INT = NBLK_PER_INT * NSPEC_PER_BLK

def calc_tspec(f_clk, NSAMP):
    '''Calculate the time for one spectrum (i.e. MCNT) in the correlator.'''
    return 1 / f_clk * NSAMP

def calc_mcnt(t, t_sync, f_clk=500e6, NSAMP=2**14):
    '''Calculate what value the correlator would have for MCNT,
    given a provided time of synchronization t_sync.'''
    t_mcnt = calc_tspec(f_clk, NSAMP)
    mcnt = np.floor((t - t_sync) / t_mcnt)
    return mcnt

def calc_t(mcnt, t_sync, f_clk=500e6, NSAMP=2**14):
    '''Calculate the time from a correlator MCNT and a sync time.'''
    t_mcnt = calc_tspec(f_clk, NSAMP)
    t = t_sync + mcnt * t_mcnt
    return t

def grid_index_from_mcnt(mcnt, mcnt_offset, start_index):
    '''Calculate the grid index from MCNT, as the correlator would.'''
    return (start_index + (mcnt - mcnt_offset) // MCNT_PER_INT) % NGRID

def calc_mcnt_offset(lst_sync, f_clk=500e6, NSAMP=2**14):
    '''Calculate MCNT_OFFSET for a synchronization pulse that
    happened at lst_sync.'''
    t_mcnt = calc_tspec(f_clk, NSAMP)
    i = grid_index_from_lst(lst_sync, f_clk=f_clk, NSAMP=NSAMP)
    lst_edge = (i + 1) * 2 * np.pi / NGRID # next grid boundary
    t_offset = (lst_edge - lst_sync) / (2 * np.pi) * t_sidereal
    mcnt_offset = int(round(t_offset / t_mcnt)) #% MCNT_PER_INT
    #mcnt_offset = int(np.floor(t_offset / t_mcnt)) #% MCNT_PER_INT
    return i+1, mcnt_offset

def lst_of_grid_index(i, f_clk=500e6, NSAMP=2**14):
    '''Calculate the true LST of the center of the LST bin
    indexed by i.'''
    lst = 2 * np.pi * (i + 0.5) / NGRID
    return lst % (2 * np.pi)

def grid_index_from_lst(lst, f_clk=500e6, NSAMP=2**14):
    '''Calculate the true grid index of the provided LST.'''
    i = np.floor(NGRID * lst / (2* np.pi))
    return i % NGRID

# Some basic tests
eps = 1e-7
for i in range(NGRID):
    lst = lst_of_grid_index(i)
    assert i == grid_index_from_lst(lst)
    assert i == grid_index_from_lst(2*np.pi/NGRID/2 + lst - eps)
    assert i == grid_index_from_lst(-2*np.pi/NGRID/2 + lst + eps)

In [58]:
import time

t_sync = time.time() # arbitrary
lst_sync = 2*np.pi - 0.1 # arbitrary
# walk through MCNTs at the end of a sidereal day
start_mcnt = 2629450000
mcnt = np.arange(start_mcnt, start_mcnt+500000)
t = calc_t(mcnt, t_sync) # calculate times corresponding to these MCNTs
lst_true = lst_sync + 2 * np.pi * (t - t_sync) / t_sidereal
start_index, mcnt_offset = calc_mcnt_offset(lst_sync)
i = grid_index_from_mcnt(mcnt, mcnt_offset, start_index)
# Calculate how many MCNTs are mis-binned to cross-check the time skew.
nmiss = len(np.where(i - grid_index_from_lst(lst_true) != 0)[0])
print('Skew after 1 sidereal day: %5.4f s' % (nmiss * calc_tspec(500e6, 2**14)))

Skew after 1 sidereal day: 0.0670 s


## Coda

If we wanted to futher suppress the periodicity error, we could consider changing the sample frequency of the correlator. Effectively, we would tweak the frequency by a small factor to ensure near-perfect peroidicity, as derived below.

In [63]:
t_error = t_sidereal - t_blk * NGRID * NBLK_PER_INT
print('Sample Clk [Hz]\tPeriodicity Error [s]')
print('%15.2f\t%+21.6f' % (f_clk, t_error))

f_clk_ideal = f_clk * (1 - t_error / t_sidereal)
t_spec_ideal = 1 / f_clk_ideal * NSAMP
t_blk_ideal = t_spec_ideal * NSPEC_PER_BLK
t_error_ideal = t_sidereal - t_blk_ideal * NGRID * NBLK_PER_INT
print('%15.2f\t%+21.6f' % (f_clk_ideal, t_error_ideal))

Sample Clk [Hz]	Periodicity Error [s]
   500000000.00	            -0.066997
   500000388.78	            +0.000000


In [7]:
# CODE LIFTED FROM casperfpga.synth.Synth
import fractions as _frac
synth_mhz = 500.00033878 # MHz 
#synth_mhz = 500.021417777777777777 # MHz 
ref_signal = 10 # MHz

# f_out = f_osc * (PLL_N + PLL_NUM/PLL_DEN) / VCO_DIV

vco_min = 1900; vco_max = 3800
if synth_mhz > vco_min and synth_mhz < vco_max:
    # Bypass VCO_DIV by properly setting OUTA_MUX and OUTB_MUX
    VCO_DIV = None
else:
    vco_guess = int(vco_min / synth_mhz) + 1
    VCO_DIV = vco_guess + vco_guess%2

# Get PLLN, PLL_NUM, and PLL_DEN
pll = float(1 if VCO_DIV is None else VCO_DIV) * synth_mhz / ref_signal
PLL_N = int(pll)
frac = pll - PLL_N
if frac < 1.0/(1<<22): # smallest fraction on the synth
    PLL_NUM = 0
    PLL_DEN = 100
else:
    fraction = _frac.Fraction(frac).limit_denominator(1<<22)
    PLL_NUM = fraction.numerator
    PLL_DEN = fraction.denominator

print(PLL_N, PLL_NUM, PLL_DEN, VCO_DIV)
# f_out = f_osc * (PLL_N + PLL_NUM/PLL_DEN) / VCO_DIV
print(ref_signal * (PLL_N + PLL_NUM/PLL_DEN) / VCO_DIV)

200 533 3933231 4
500.00033878000045
